---
# Algoritmos para Big Data

**Handout 2 -  Data joining, windowing, and Spark SQL**

**2024/25**

This lab class aims to get hands-on experience on three issues related to data processing: data joining, data windowing and Spark SQL.

This notebook should contain the implementation of the tasks presented in the handout.

Hence both handout and notebook must be considered together as one.

---
# Task A - Data ingestion

**Datasest**

The file can be downloaded from

https://bigdata.iscte-iul.eu/datasets/retail-data.csv

**Spark setup**

In [1]:
# Basic imports
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
# Build SparkSession
spark = SparkSession.builder.appName("JoinWindowingSQL").getOrCreate()

**Reading and checking data**

In [8]:
# Reading data
# data_dir ='../../Datasets/'
file_retail = 'retail-data.csv'

! head $file_retail
df_retail = spark.read.csv(file_retail, header=True, sep=';', inferSchema=True)


InvoiceNo;StockCode;Description;Quantity;InvoiceDate;UnitPrice;CustomerID;Country
536375;71053;WHITE METAL LANTERN;6;01/12/2010 09:32;3.39;17850;United Kingdom
C536391;21983;PACK OF 12 BLUE PAISLEY TISSUES ;-24;01/12/2010 10:24;0.29;17548;United Kingdom
536395;21314;SMALL GLASS HEART TRINKET POT;8;01/12/2010 10:47;2.1;13767;United Kingdom
536396;82494L;WOODEN FRAME ANTIQUE WHITE ;12;01/12/2010 10:51;2.55;17850;United Kingdom
536425;22837;HOT WATER BOTTLE BABUSHKA ;8;01/12/2010 12:08;4.65;13758;United Kingdom
536464;20878;SET/9 CHRISTMAS T-LIGHTS SCENTED ;1;01/12/2010 12:23;1.25;17968;France
536520;22760;TRAY, BREAKFAST IN BED;1;01/12/2010 12:43;12.75;14729;United Kingdom
536520;22812;PACK 3 BOXES CHRISTMAS PANNETONE;3;01/12/2010 12:43;1.95;14729;United Kingdom
536530;21071;VINTAGE BILLBOARD DRINK ME MUG;24;01/12/2010 13:21;1.25;17905;France


In [10]:
df_retail.show(10)
print(f'df_retail - number of rows is {df_retail.count()}.')
df_retail.printSchema()

+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+----------------+---------+----------+--------------+
|   536375|    71053| WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|     17850|United Kingdom|
|  C536391|    21983|PACK OF 12 BLUE P...|     -24|01/12/2010 10:24|     0.29|     17548|United Kingdom|
|   536395|    21314|SMALL GLASS HEART...|       8|01/12/2010 10:47|      2.1|     13767|United Kingdom|
|   536396|   82494L|WOODEN FRAME ANTI...|      12|01/12/2010 10:51|     2.55|     17850|United Kingdom|
|   536425|    22837|HOT WATER BOTTLE ...|       8|01/12/2010 12:08|     4.65|     13758|United Kingdom|
|   536464|    20878|SET/9 CHRISTMAS T...|       1|01/12/2010 12:23|     1.25|     17968|        France|
|   536520|    22760|TRAY, BREAKFAST I...|       1|01/1

In [11]:
print(f'df_retail - number of rows is {df_retail.count()  }; after dropDuplicates() applied would be {df_retail.dropDuplicates().count()   }.')

df_retail - number of rows is 3054; after dropDuplicates() applied would be 3054.


In [12]:
print(f'''df_retail - number of rows after dropna(how='any') would be {df_retail.dropna(how='any')     }.''')

df_retail - number of rows after dropna(how='any') would be DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string].


In [13]:
print('Checking nulls at each column of df_retail')
dict_nulls_retail = {col: df_retail.filter(df_retail[col].isNull()).count() for col in df_retail.columns}
dict_nulls_retail

Checking nulls at each column of df_retail


{'InvoiceNo': 0,
 'StockCode': 0,
 'Description': 0,
 'Quantity': 0,
 'InvoiceDate': 0,
 'UnitPrice': 0,
 'CustomerID': 0,
 'Country': 0}

Data seems fine!

---
# Task B - Data joining

Combining rows from two DataFrames, based on one common column. Types of operation to consider are:
- Inner join
- Left join
- Full join
- Cross join
- Anti join

Based on df_retail, three DataFrames will be created:
- df_sales, as df_retail but excluding column Country
- df_customers, with two columns: CustomerID and Country
- df_customers_nonUK, likewise df_costumers but filtering out the customers from United Kingdom


Common column as condition will be CustomerID.


## 1.

In [16]:
# df_retail.groupBy('InvoiceNo','Country').count().show()
df_retail.groupBy('InvoiceNo','Country').count().groupBy('Country').count().show()

+--------------+-----+
|       Country|count|
+--------------+-----+
|       Germany|    4|
|        France|    9|
|          EIRE|    3|
|        Norway|    2|
|     Australia|    5|
|United Kingdom|  117|
|   Netherlands|    2|
+--------------+-----+



## 2.

In [18]:
# As df_retail but excluding the column Country
df_sales = df_retail.drop('Country')
df_sales.show()

+---------+---------+--------------------+--------+----------------+---------+----------+
|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|
+---------+---------+--------------------+--------+----------------+---------+----------+
|   536375|    71053| WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|     17850|
|  C536391|    21983|PACK OF 12 BLUE P...|     -24|01/12/2010 10:24|     0.29|     17548|
|   536395|    21314|SMALL GLASS HEART...|       8|01/12/2010 10:47|      2.1|     13767|
|   536396|   82494L|WOODEN FRAME ANTI...|      12|01/12/2010 10:51|     2.55|     17850|
|   536425|    22837|HOT WATER BOTTLE ...|       8|01/12/2010 12:08|     4.65|     13758|
|   536464|    20878|SET/9 CHRISTMAS T...|       1|01/12/2010 12:23|     1.25|     17968|
|   536520|    22760|TRAY, BREAKFAST I...|       1|01/12/2010 12:43|    12.75|     14729|
|   536520|    22812|PACK 3 BOXES CHRI...|       3|01/12/2010 12:43|     1.95|     14729|
|   536530

In [19]:
# After df_retail but with just two columns: CustomerID and Country
df_customers = df_retail.select('CustomerID', 'Country').distinct().orderBy('Country')
df_customers.groupBy('Country').count().show()

+--------------+-----+
|       Country|count|
+--------------+-----+
|       Germany|    4|
|        France|   15|
|          EIRE|    2|
|        Norway|    2|
|     Australia|    4|
|United Kingdom|  100|
|   Netherlands|    2|
+--------------+-----+



In [20]:
# Likewise df_customers but filtering out the customers from United Kingdom
df_customers_nonUK = df_customers.filter(F.col('Country') != 'United Kingdom')    
print(f'df_customers_nonUK - number of rows is {df_customers_nonUK.count()}.')

df_customers_nonUK - number of rows is 29.


## 3.

In [21]:
# Inner join
df = df_sales.join(df_customers_nonUK, on='CustomerID',how='inner')
df.show(10)
print(f'Inner join - number of rows is {df.count()}.')

+----------+---------+---------+--------------------+--------+----------------+---------+---------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|  Country|
+----------+---------+---------+--------------------+--------+----------------+---------+---------+
|     17968|   536464|    20878|SET/9 CHRISTMAS T...|       1|01/12/2010 12:23|     1.25|   France|
|     17905|   536530|    21071|VINTAGE BILLBOARD...|      24|01/12/2010 13:21|     1.25|   France|
|     17905|   536530|    22943|CHRISTMAS LIGHTS ...|       2|01/12/2010 13:21|     4.95|   France|
|     12433|   536532|    22534|MAGIC DRAWING SLA...|      24|01/12/2010 13:24|     0.42|   Norway|
|     22578|   536592|    22472|TV DINNER TRAY DO...|       3|01/12/2010 17:06|    11.02|  Germany|
|     22578|   536592|    22620|4 TRADITIONAL SPI...|       2|01/12/2010 17:06|     2.51|  Germany|
|     19997|   536592|   84536B|FAIRY CAKES NOTEB...|       1|01/12/2010 17:06|     0.85|Australia|


In [22]:
# Left join
df = df_sales.join(df_customers_nonUK, on='CustomerID',how='left')
df.show(10)
print(f'Left join - number of rows is {df.count()}.')

+----------+---------+---------+--------------------+--------+----------------+---------+-------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|Country|
+----------+---------+---------+--------------------+--------+----------------+---------+-------+
|     17850|   536375|    71053| WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|   NULL|
|     17548|  C536391|    21983|PACK OF 12 BLUE P...|     -24|01/12/2010 10:24|     0.29|   NULL|
|     13767|   536395|    21314|SMALL GLASS HEART...|       8|01/12/2010 10:47|      2.1|   NULL|
|     17850|   536396|   82494L|WOODEN FRAME ANTI...|      12|01/12/2010 10:51|     2.55|   NULL|
|     13758|   536425|    22837|HOT WATER BOTTLE ...|       8|01/12/2010 12:08|     4.65|   NULL|
|     17968|   536464|    20878|SET/9 CHRISTMAS T...|       1|01/12/2010 12:23|     1.25| France|
|     14729|   536520|    22760|TRAY, BREAKFAST I...|       1|01/12/2010 12:43|    12.75|   NULL|
|     14729|   53652

In [23]:
# Full join
df = df_sales.join(df_customers_nonUK, on='CustomerID',how='full')
df.show(10)
print(f'Full join - number of rows is {df.count()}.')

+----------+---------+---------+--------------------+--------+----------------+---------+---------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|  Country|
+----------+---------+---------+--------------------+--------+----------------+---------+---------+
|     12431|   536389|    22941|CHRISTMAS LIGHTS ...|       6|01/12/2010 10:03|      8.5|Australia|
|     12431|   536389|    22193|RED DINER WALL CLOCK|       2|01/12/2010 10:03|      8.5|Australia|
|     12431|   536389|    21791|VINTAGE HEADS AND...|      12|01/12/2010 10:03|     1.25|Australia|
|     12431|   536389|    22726|ALARM CLOCK BAKEL...|       4|01/12/2010 10:03|     3.75|Australia|
|     12431|   536389|    22727|ALARM CLOCK BAKEL...|       4|01/12/2010 10:03|     3.75|Australia|
|     12431|   536389|    22195|LARGE HEART MEASU...|      24|01/12/2010 10:03|     1.65|Australia|
|     12431|   536389|   35004G|SET OF 3 GOLD FLY...|       4|01/12/2010 10:03|     6.35|Australia|


In [24]:
# Cross join
spark.conf.set("spark.sql.crossJoin.enabled", "true")

df = df_sales.crossJoin(df_customers_nonUK)
df.show(10)
print(f'Cross join - number of rows is {df.count()}.')


+---------+---------+-------------------+--------+----------------+---------+----------+----------+-----------+
|InvoiceNo|StockCode|        Description|Quantity|     InvoiceDate|UnitPrice|CustomerID|CustomerID|    Country|
+---------+---------+-------------------+--------+----------------+---------+----------+----------+-----------+
|   536375|    71053|WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|     17850|     19588|Netherlands|
|   536375|    71053|WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|     17850|     18444|     France|
|   536375|    71053|WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|     17850|     19888|  Australia|
|   536375|    71053|WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|     17850|     20145|  Australia|
|   536375|    71053|WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|     17850|     21365|    Germany|
|   536375|    71053|WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|     17850|     17873|     

In [26]:
# Anti join
df = df_sales.join(df_customers_nonUK, on='CustomerID',how='anti')
df.show(10)
print(f'Anti join - number of rows is {df.count()}.')

+----------+---------+---------+--------------------+--------+----------------+---------+
|CustomerID|InvoiceNo|StockCode|         Description|Quantity|     InvoiceDate|UnitPrice|
+----------+---------+---------+--------------------+--------+----------------+---------+
|     17850|   536375|    71053| WHITE METAL LANTERN|       6|01/12/2010 09:32|     3.39|
|     17548|  C536391|    21983|PACK OF 12 BLUE P...|     -24|01/12/2010 10:24|     0.29|
|     13767|   536395|    21314|SMALL GLASS HEART...|       8|01/12/2010 10:47|      2.1|
|     17850|   536396|   82494L|WOODEN FRAME ANTI...|      12|01/12/2010 10:51|     2.55|
|     13758|   536425|    22837|HOT WATER BOTTLE ...|       8|01/12/2010 12:08|     4.65|
|     14729|   536520|    22760|TRAY, BREAKFAST I...|       1|01/12/2010 12:43|    12.75|
|     14729|   536520|    22812|PACK 3 BOXES CHRI...|       3|01/12/2010 12:43|     1.95|
|     19224|   536544|    22812|PACK 3 BOXES CHRI...|       2|01/12/2010 14:32|     4.21|
|     1865

---
# Task C - Data windowing

In relation to a column value (quantity x unit price), and considering a window with some size, how to compute:
- a new column whose value is equal to the sum of the current row and previous rows?
- the lag?

## 1.

In [27]:
# Add value column as quantity * unit price, and then ordered by invoice date
df_invoice_value = ( df_retail
             .withColumn('Value', F.col('Quantity')*F.col('UnitPrice'))
             .orderBy("InvoiceDate", ascending = True)
             .select('InvoiceDate', 'CustomerID', 'Country', 'Value')
)
df_invoice_value.show()



+----------------+----------+--------------+------------------+
|     InvoiceDate|CustomerID|       Country|             Value|
+----------------+----------+--------------+------------------+
|01/12/2010 08:26|     17850|United Kingdom|              15.3|
|01/12/2010 08:26|     17850|United Kingdom|             20.34|
|01/12/2010 08:26|     17850|United Kingdom|15.299999999999999|
|01/12/2010 08:26|     17850|United Kingdom|              25.5|
|01/12/2010 08:26|     17850|United Kingdom|              22.0|
|01/12/2010 08:26|     17850|United Kingdom|             20.34|
|01/12/2010 08:26|     17850|United Kingdom|             20.34|
|01/12/2010 08:28|     17850|United Kingdom|11.100000000000001|
|01/12/2010 08:28|     17850|United Kingdom|11.100000000000001|
|01/12/2010 08:34|     13047|United Kingdom|             17.85|
|01/12/2010 08:34|     13047|United Kingdom|              30.0|
|01/12/2010 08:34|     13047|United Kingdom|              31.8|
|01/12/2010 08:34|     13047|United King

## 2.

In [28]:
from pyspark.sql import Window

# Show a new column related to Value that, for each  invoice/row, 
# would contain the sum of n previous rows (of column Value)
window_previous = -2
window = Window.partitionBy('Country', 'CustomerID').orderBy('CustomerID', 'InvoiceDate').rowsBetween(window_previous, 0)
# 'CustomerID', 'InvoiceDate', ascending=[True, True]
df_invoice_value.withColumn('SumLastSalesForCustomer',F.sum('Value').over(window)).show()

+----------------+----------+---------+------------------+-----------------------+
|     InvoiceDate|CustomerID|  Country|             Value|SumLastSalesForCustomer|
+----------------+----------+---------+------------------+-----------------------+
|01/12/2010 10:03|     12431|Australia|              51.0|                   51.0|
|01/12/2010 10:03|     12431|Australia|              17.0|                   68.0|
|01/12/2010 10:03|     12431|Australia|              15.0|                   83.0|
|01/12/2010 10:03|     12431|Australia|              15.0|                   47.0|
|01/12/2010 10:03|     12431|Australia|              15.0|                   45.0|
|01/12/2010 10:03|     12431|Australia|39.599999999999994|                   69.6|
|01/12/2010 10:03|     12431|Australia|              25.4|                   80.0|
|01/12/2010 10:03|     12431|Australia|              35.7|                  100.7|
|01/12/2010 10:03|     12431|Australia|             17.85|                  78.95|
|01/

## 3.

In [29]:
# Show a new column related to Value that, for each invoice/row, 
# would contain the value corresponding to a lag of some size (of column Value)
window = Window.partitionBy('Country', 'CustomerID').orderBy('CustomerID', 'InvoiceDate')
window_size = 2
df_invoice_value.withColumn('PreviousNValueForCustomer',F.lag('Value', window_size).over(window)).show()

+----------------+----------+---------+------------------+-------------------------+
|     InvoiceDate|CustomerID|  Country|             Value|PreviousNValueForCustomer|
+----------------+----------+---------+------------------+-------------------------+
|01/12/2010 10:03|     12431|Australia|              51.0|                     NULL|
|01/12/2010 10:03|     12431|Australia|              17.0|                     NULL|
|01/12/2010 10:03|     12431|Australia|              15.0|                     51.0|
|01/12/2010 10:03|     12431|Australia|              15.0|                     17.0|
|01/12/2010 10:03|     12431|Australia|              15.0|                     15.0|
|01/12/2010 10:03|     12431|Australia|39.599999999999994|                     15.0|
|01/12/2010 10:03|     12431|Australia|              25.4|                     15.0|
|01/12/2010 10:03|     12431|Australia|              35.7|       39.599999999999994|
|01/12/2010 10:03|     12431|Australia|             17.85|       

---
# Task D - Spark SQL

SQL operations to work with:
- Storing the retail data as a persistent table
- Use of SQL expressions to query the table, similarly to a DataFrame
- Creating a temporary table and then as above query it

## 1.

In [30]:
# Save the retail data as persistent table into the Hive metastore
df_retail.write.mode('overwrite').saveAsTable('RetailTable')


## 2.

In [31]:
# List of databases
print(spark.catalog.listDatabases())

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/home/jovyan/code/aula_3/spark-warehouse')]


In [32]:
# List of tables in the default database
spark.catalog.listTables(dbName='default')

[Table(name='retailtable', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False)]

In [33]:
# List of columns in the retail table
spark.catalog.listColumns('retailtable')

[Column(name='InvoiceNo', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='StockCode', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='Description', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='Quantity', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='InvoiceDate', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='UnitPrice', description=None, dataType='double', nullable=True, isPartition=False, isBucket=False),
 Column(name='CustomerID', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='Country', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

## 3.

In [34]:
# Use of the default managed table
spark.sql('USE default')

DataFrame[]

In [ ]:
# Show the content of retail table
spark.sql('SELECT * FROM RetailTable').show(truncate = False)

+---------+---------+-----------------------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate     |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+----------------+---------+----------+--------------+
|536375   |71053    |WHITE METAL LANTERN                |6       |01/12/2010 09:32|3.39     |17850     |United Kingdom|
|C536391  |21983    |PACK OF 12 BLUE PAISLEY TISSUES    |-24     |01/12/2010 10:24|0.29     |17548     |United Kingdom|
|536395   |21314    |SMALL GLASS HEART TRINKET POT      |8       |01/12/2010 10:47|2.1      |13767     |United Kingdom|
|536396   |82494L   |WOODEN FRAME ANTIQUE WHITE         |12      |01/12/2010 10:51|2.55     |17850     |United Kingdom|
|536425   |22837    |HOT WATER BOTTLE BABUSHKA          |8       |01/12/2010 12:08|4.65     |13758     |United Kingdom|
|536464   |20878    |SET/9 CHRISTMAS T-L

## 4.

In [37]:
# Create a temporary table with just the customers
df_customers.createOrReplaceTempView('CustomersTable')

In [38]:
# CustomerID and country in all rows, with rows order by customerID
spark.sql('SELECT CustomerID, Country FROM RetailTable').orderBy('CustomerID').show()

+----------+---------+
|CustomerID|  Country|
+----------+---------+
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12431|Australia|
|     12433|   Norway|
|     12433|   Norway|
|     12433|   Norway|
|     12433|   Norway|
|     12433|   Norway|
|     12433|   Norway|
+----------+---------+
only showing top 20 rows



In [44]:
# Counting of rows regarding each country registered
spark.sql('SELECT Country, count(*) as count FROM RetailTable GROUP BY Country').orderBy('Country').show()

+--------------+-----+
|       Country|count|
+--------------+-----+
|     Australia|  111|
|          EIRE|   48|
|        France|  490|
|       Germany|  116|
|   Netherlands|   32|
|        Norway|   95|
|United Kingdom| 2162|
+--------------+-----+

